In [17]:
import os.path
import textwrap
import requests
import pandas as pd

def generate_sparql_query(fullName, property_labels_to_ids, language='en'):
    """
    Query WikiData for the properties of the given person listed in the given property map.
    All properties that are simple values without a label must have an "_id" suffix, all date
    properties must begin with "date"
    :param fullName: 
    :param property_labels_to_ids: 
    :param language: 
    :return: 
    """
    propSelection = ""
    for label, pid in property_labels_to_ids.items():
        if label.endswith("_id") or label.startswith("image"):
            # literal values, including URIs - this needs to be solved in a more generic way
            propSelection += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}. }}"""
        elif label.startswith("date"):  
            # Dates, fetched directly but need special handling for formatting if desired
            propSelection += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}. }}"""
        else:
            propSelection += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}Id .
                   ?{label}Id rdfs:label ?{label} FILTER(LANG(?{label}) = "{language}") .
                   SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{language}". }} }}"""

    query = textwrap.dedent(f"""
    SELECT DISTINCT ?item ?itemLabel {"".join([f"(SAMPLE(?{label}) AS ?{label})" for label in property_labels_to_ids])}
    WHERE {{
          ?item wdt:P31 wd:Q5; rdfs:label "{fullName}"@{language}.
          {textwrap.dedent(propSelection)}
    }}
    GROUP BY ?item ?itemLabel 
    """)
    return query

def construct_image_url(filename):
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{requests.utils.quote(filename)}"



def query_wikidata(fullName, property_map, language='en'):
    SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
    query = generate_sparql_query(fullName, property_map, language)
    headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/json'}
    response = requests.get(SPARQL_ENDPOINT, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code != 200:
        response.raise_for_status()
        
    results = response.json()['results']['bindings']
    
    if not results:
        return []

    for i, result in enumerate(results):
        # Initialize with fullName to ensure it appears first
        data = {
            'fullName': fullName
        }
   
        for label in property_map:
            if label in result:
                value = result[label]['value']
                data[label] = value
            else:
                data[label] = None
                
        # add item URI
        data['qid'] = os.path.basename(result['item']['value'])
        results[i] = data
            
    return results

def get_wikipedia_links(qid, languages):
    """
    Fetch Wikipedia links for a given Wikidata QID and a list of languages.

    Parameters:
    - qid (str): The QID of the Wikidata item.
    - languages (list): A list of language codes (e.g., ['en', 'de']).

    Returns:
    - dict: A dictionary with languages as keys and Wikipedia URLs as values.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "sitelinks/urls",
        "format": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()

    links = {}
    if "entities" in data and qid in data["entities"]:
        sitelinks = data["entities"][qid].get("sitelinks", {})
        for lang in languages:
            sitekey = f"{lang}wiki"
            if sitekey in sitelinks:
                # Use the 'title' key and construct the URL manually
                title = sitelinks[sitekey]["title"]
                links[lang] = f"https://{lang}.wikipedia.org/wiki/{requests.utils.quote(title)}"
            else:
                links[lang] = None  # Or use '' to represent absence of link

    return links



def get_person_info_from_wikidata(names, property_map, language='en'):
    all_data = []
    for fullName in names:
        results = query_wikidata(fullName, property_map, language)
        all_data += results
    if len(all_data) > 0:
        # Ensure fullName appears first by reordering columns based on property_labels_to_ids keys
        columns_order = ['fullName', 'qid'] + list(property_map.keys())
        df = pd.DataFrame(all_data, columns=columns_order)
        # add wikipedia links
        df[f'wikipedia_{language}'] = df.apply(lambda r: get_wikipedia_links(r['qid'], [language]).get(language), axis=1 )
    else:
        df = pd.DataFrame(columns=['fullName'] + list(property_map.keys()) + [f'wikipedia_{language}']) 
    return df

In [19]:
# Now calling the updated function with the 'language' parameter
property_labels_to_ids = {
    'sexOrGender': 'P21',
#    'image': 'P18',
    'countryOfCitizenship': 'P27',
    'givenName': 'P735',
    'familyName': 'P734',
    'dateOfBirth': 'P569',
    'dateOfDeath': 'P570',
    'occupation': 'P106',
    'fieldOfWork': 'P101',
    'viaf_id': 'P214',
    'isni_id': 'P213',
    'gnd_id': 'P227'
}

scholars = [
    "Hans Kelsen",
    "Hugo Sinzheimer",
    "Karl Renner",
    "Ernst Fraenkel",
    "Franz Leopold Neumann",
    "Otto Kahn-Freund",
    "Otto Kirchheimer",
    "Herrmann Kantorowicz",
    "Ludwig Bendix",
    "Arthur Nussbaum",
    "Theodor Geiger",
    "Erhard Blankenburg",
    "Wolfgang Kaupen",
    "Rüdiger Lautmann",
    "Thilo Ramm",
    "Rudolf Wiethölter",
    "Niklas Luhmann",
    "Gunther Teubner",
    "Volkmar Gessner",
    "Konstanze Plett",
    "Ute Sacksofsky",
    "Susanne Baer"
]
df = get_person_info_from_wikidata(scholars, property_labels_to_ids)
df

,fullName,qid,sexOrGender,image,countryOfCitizenship,givenName,familyName,dateOfBirth,dateOfDeath,occupation,fieldOfWork,viaf_id,isni_id,gnd_id,wikipedia_en
0,Hans Kelsen,Q84165,male,http://commons.wikimedia.org/wiki/Special:File...,United States of America,Hans,Kelsen,1881-10-11T00:00:00Z,1973-04-19T00:00:00Z,philosopher,constitutional law,31998356,0000000121266076,118561219,https://en.wikipedia.org/wiki/Hans%20Kelsen
1,Hugo Sinzheimer,Q86043,male,http://commons.wikimedia.org/wiki/Special:File...,Germany,D.,Sinzheimer,1875-01-01T00:00:00Z,1945-09-16T00:00:00Z,trade unionist,None,27864307,0000000109619641,118614711,https://en.wikipedia.org/wiki/Hugo%20Sinzheimer
2,Karl Renner,Q48010797,male,None,Austria,Karl,Renner,1884-12-24T00:00:00Z,1929-06-24T00:00:00Z,None,None,78817437,0000000054940875,134869036,None
3,Karl Renner,Q15451414,male,http://commons.wikimedia.org/wiki/Special:File...,Germany,Karl,Renner,1833-05-13T00:00:00Z,1913-09-22T00:00:00Z,merchant,None,171170593,None,1012296458,None
4,Karl Renner,Q94817885,male,None,None,Karl,Renner,None,None,board member,None,100408035,None,139099409,None
5,Karl Renner,Q11726,male,http://commons.wikimedia.org/wiki/Special:File...,Austria,Karl,Renner,1870-12-14T00:00:00Z,1950-12-31T00:00:00Z,diplomat,politics,61669459,0000000121358165,118599739,https://en.wikipedia.org/wiki/Karl%20Renner
6,Karl Renner,Q107022720,male,None,None,Karl,Renner,1889-12-01T00:00:00Z,1947-05-17T00:00:00Z,local politician,None,None,None,None,None
7,Ernst Fraenkel,Q21997821,male,None,None,Ernst,Fraenkel,1923-01-01T00:00:00Z,2014-11-13T00:00:00Z,businessperson,None,None,None,None,https://en.wikipedia.org/wiki/Ernst%20Fraenkel...
8,Ernst Fraenkel,Q92522,male,None,Germany,Ernst,Fraenkel,1881-10-16T00:00:00Z,1957-10-02T00:00:00Z,linguist,linguistics,24595835,0000000081027854,11669064X,https://en.wikipedia.org/wiki/Ernst%20Fraenkel...
9,Ernst Fraenkel,Q1358093,male,None,Germany,Ernst,Fraenkel,1891-04-05T00:00:00Z,1971-08-18T00:00:00Z,economic historian,None,50078162,None,121259854,None


In [ ]:
from lib.dnb_data import 

In [21]:
df.to_csv("scholars.csv", index=False)

In [4]:
import pandas as pd

# Load the data
df = pd.read_csv("scholars.csv", encoding='utf-8')
# Convert date strings to datetime
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'])
df['dateOfDeath'] = pd.to_datetime(df['dateOfDeath'])

# Create a new column for the duration
df['duration'] = df['dateOfDeath'] - df['dateOfBirth']

# Display the first few rows to check
df.head()


,fullName,item,sexOrGender,image,countryOfCitizenship,givenName,familyName,dateOfBirth,dateOfDeath,occupation,fieldOfWork,employer,viaf_id,isni_id,gnd_id,duration
0,Hans Kelsen,Q84165,male,http://commons.wikimedia.org/wiki/Special:File...,Cisleithania,Hans,Kelsen,1881-10-11 00:00:00+00:00,1973-04-19 00:00:00+00:00,judge,international law,Charles University,31998356,0000000121266076,118561219.0,33427 days
1,Hugo Sinzheimer,Q86043,male,http://commons.wikimedia.org/wiki/Special:File...,Germany,Hugo,Sinzheimer,1875-04-12 00:00:00+00:00,1945-09-16 00:00:00+00:00,lawyer,NaN,Goethe University Frankfurt,27864307,0000000109619641,118614711.0,25724 days
2,Karl Renner,Q11726,male,http://commons.wikimedia.org/wiki/Special:File...,Cisleithania,Karl,Renner,1870-12-14 00:00:00+00:00,1950-12-31 00:00:00+00:00,lawyer,politics,Austrian Federal Government,61669459,0000000121358165,118599739.0,29236 days
3,Ernst Fraenkel,Q86812,male,NaN,Germany,Ernst,Fraenkel,1898-12-26 00:00:00+00:00,1975-03-28 00:00:00+00:00,lawyer,NaN,Free University Berlin,27108403,0000000110230959,118534602.0,27850 days
4,Franz Leopold Neumann,Q112562068,male,NaN,NaN,Leopold,Neumann,NaT,NaT,printer,publishing,NaN,637163874508945722514,NaN,NaN,NaT


In [8]:
import pandas as pd
import plotly.express as px

# Assuming df is already loaded and dateOfBirth/dateOfDeath are converted to datetime

# Initialize a list to track the last dateOfDeath in each row to manage overlaps
last_dates = []

# Function to find the appropriate row for each scholar
def find_row(last_dates, start_date):
    for i, last_date in enumerate(last_dates):
        if start_date > last_date:
            return i
    return len(last_dates)

# Assign rows without overlaps and sort by the earliest dateOfBirth
df['row'] = 0
for index, scholar in df.iterrows():
    row = find_row(last_dates, scholar['dateOfBirth'])
    if row < len(last_dates):
        last_dates[row] = scholar['dateOfDeath']
    else:
        last_dates.append(scholar['dateOfDeath'])
    df.at[index, 'row'] = row

# Now plotting without row labels
fig = px.timeline(df, x_start="dateOfBirth", x_end="dateOfDeath", y="row", text="fullName", title="Scholars' Life Spans Timeline")

# Update layout
fig.update_layout(yaxis=dict(tickmode='array', tickvals=[], ticktext=[]))
fig.update_yaxes(autorange="reversed")  # This reverses the y-axis to match your requirement

fig.show()



In [44]:
import pandas as pd
from datetime import datetime

# Assuming df is your existing DataFrame

# Convert dateOfBirth and dateOfDeath to just the year, handle NaT/NaN appropriately
df['Year'] = pd.to_datetime(df['dateOfBirth'], errors='coerce').dt.year.astype('Int64')
df['End Year'] = pd.to_datetime(df['dateOfDeath'], errors='coerce').dt.year.astype('Int64')

# Create 'Display Date' as "dateOfBirth - dateOfDeath"
df['Display Date'] = df['Year'].astype(str).replace('<NA>','')  + ' - ' + df['End Year'].astype(str).replace('<NA>','')

# Create 'Headline' as "fullName (dateOfBirth - dateOfDeath)"
df['Headline'] = df['fullName'] + ' (' + df['Display Date'] + ')'

# Create 'Text' column by combining occupation, fieldOfWork, employer
df['Text'] = df[['occupation', 'fieldOfWork']].apply(lambda x: '<br>'.join(x.dropna()), axis=1)

# Use the image directly; assuming the URLs are already correctly formed in the 'image' column
df['Media'] = df['image']

# Add a "Group" column with the value "actors" for all rows
df['Group'] = 'actors'

# fix date columns
df['Display Date'] = df['Display Date'].fillna('')  # Ensure no NaNs in Display Date
df['Headline'] = df['Headline'].fillna('')  # Ensure no NaNs in Headline
df['Text'] = df['Text'].fillna('')  # Ensure no NaNs in Text
df['Media'] = df['Media'].fillna('')  # Ensure no NaNs in Media

# Now select and order the DataFrame according to the TimelineJS template requirements
columns = "Year	Month	Day	Time	End Year	End Month	End Day	End Time	Display Date	Headline	Text	Media	Media Credit	Media Caption	Media Thumbnail	Type	Group	Background	Link".split("\t")
for col in columns:
    if col not in df:
        df[col] = ''
timeline_df = df[columns]

timeline_df.to_excel("timeline_data.xlsx", index=False)
